# Testing the RAG agent

The objective on this notebook is to test the RAG agent to identify prices!

The previous notebook can be found: [`experiments/agent.ipynb`](agent.ipynb).

This notebook will test:
* Context: 
    * Precision;
    * Recall. 
* Faithfulness
* Medium latency;

---

Importing packages:

In [1]:
!pip install ragas rapidfuzz pandas

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_ollama import ChatOllama
from ollama import Client

from pandas import DataFrame

from ragas.llms import LangchainLLMWrapper
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate
from ragas.metrics import (
    Faithfulness, 
    AnswerCorrectness,
    AnswerRelevancy,
    ContextRecall,
    ContextPrecision,
    NonLLMContextPrecisionWithReference,
    NonLLMContextRecall
    )

from ragas.config import run

from time import time
from typing import TypedDict
from json import load

In [3]:
%cd ..

/home/gabael/projetos/precificador


In [37]:
from src.ollama.llm import chat
from src.ollama.embedding import embeddings

from src.graph.rag import generate_price
from src.rag.rag import generate_vector_store, generate_documents_from_web

---

Prepare test environment:

In [ ]:
OLLAMA_URL = "http://localhost:11434"
EVALUATE_MODEL = "llama3-2:1b"

In [6]:
client = Client(OLLAMA_URL)

In [7]:
client.pull(EVALUATE_MODEL)

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [8]:
evaluate_chat = ChatOllama(base_url=OLLAMA_URL, model=EVALUATE_MODEL)

In [9]:
class Question(TypedDict):
    query: str
    product: str

In [10]:
with open("dataset/questions.json", 'r') as f:
    questions: list[Question] = load(f)

In [11]:
products = set(q['product'] for q in questions)

In [12]:
documents = {
    product: generate_documents_from_web(product)
    for product in products
    }

In [13]:
vector_stores = {
    product: generate_vector_store(documents[product])
    for product in products
}

In [14]:
retrievals = [
    vector_stores[question['product']].similarity_search(question['query'], 10)
    for question in questions
]

In [15]:
def break_line() -> str:
    return '\n'

answers = [
    chat.invoke([
        SystemMessage(f"""Você é um assistente especializado em realizar pesquisas de preço. Responda apenas com o preço e onde comprar.
                                  
                        Utilize esta base de conhecimento: {
                            break_line().join([
                                doc.page_content for doc in retrieval
                            ])}."""),
        HumanMessage(question['query'])])
    for retrieval, question in zip(retrievals, questions)
]

In [16]:
single_samples = [
    SingleTurnSample(
        user_input=question['query'],
        response=str(answer.content),
        reference=question['query'],
        retrieved_contexts=[r.page_content for r in retrieval],
        reference_contexts=[doc.page_content for doc in documents[question['product']]]
        ) 
    for (question, retrieval, answer) 
    in zip(questions, retrievals, answers)
]

In [33]:
dataset = EvaluationDataset(single_samples)

In [34]:
#llm = LangchainLLMWrapper(evaluate_chat)
llm = LangchainLLMWrapper(chat)

---

Evaluate:

In [40]:
metrics = [
    Faithfulness(), 
    AnswerCorrectness(),
    AnswerRelevancy(),
    ContextRecall(),
    ContextPrecision(),
    NonLLMContextPrecisionWithReference(),
    NonLLMContextRecall()
]

In [41]:
score = evaluate(
    dataset=dataset,
    metrics=metrics,
    llm=llm,
    embeddings=embeddings,
    raise_exceptions=False,
)

Evaluating:   5%|▌         | 7/140 [04:23<1:50:08, 49.68s/it]Exception raised in Job[14]: TimeoutError()
Exception raised in Job[16]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[18]: TimeoutError()
Exception raised in Job[7]: TimeoutError()
Exception raised in Job[1]: TimeoutError()
Exception raised in Job[10]: TimeoutError()
Exception raised in Job[0]: TimeoutError()
Exception raised in Job[11]: TimeoutError()
Exception raised in Job[3]: TimeoutError()
Exception raised in Job[15]: TimeoutError()
Exception raised in Job[4]: TimeoutError()
Exception raised in Job[21]: TimeoutError()
Exception raised in Job[8]: TimeoutError()
Exception raised in Job[9]: TimeoutError()
Evaluating:  21%|██▏       | 30/140 [09:01<49:55, 27.23s/it] Exception raised in Job[25]: TimeoutError()
Exception raised in Job[23]: TimeoutError()
Exception raised in Job[30]: TimeoutError()
Exception raised in Job[36]: TimeoutError()
Exception raised in Job[29]: TimeoutError()
Excep

In [43]:
df = score.to_pandas()

In [44]:
df

,user_input,retrieved_contexts,reference_contexts,response,reference,faithfulness,answer_correctness,answer_relevancy,context_recall,context_precision,non_llm_context_precision_with_reference,non_llm_context_recall
0,Vou me casar em breve e quero uma geladeira. N...,[0.5% de volta na loja todaCompare entre 13 lo...,[Geladeira com o menor preço | Zoom Categorias...,"Parabéns pela sua notícia!\n\nPara um casal, é...",Vou me casar em breve e quero uma geladeira. N...,NaN,NaN,0.791330,NaN,NaN,1.0,0.097826
1,"Estou em dúvida sobre fogão, qual a marca mais...",[deve ser colocado embaixo de um fogão a lenha...,[0.5% de volta na loja todaCompare entre 16 lo...,Vou te fornecer as informações que você solici...,"Estou em dúvida sobre fogão, qual a marca mais...",NaN,NaN,NaN,NaN,NaN,1.0,0.373288
2,"Eu preciso comprar um celular pra meu noivo, q...",[de luxo para samsung galaxy a53 a73 a33 a32 a...,[Compare entre 16 lojasIndicaCelular Samsung G...,Abaixo estão alguns modelos recentes da linha ...,"Eu preciso comprar um celular pra meu noivo, q...",NaN,NaN,NaN,NaN,NaN,1.0,0.338710
3,"Gostaria de uma Máquina de lavar de 15kg, mas ...","[Consul, Electrolux são os principais fabrican...",[Máquina de lavar 15 kg com o menor preço | Zo...,Entendo sua preocupação! Uma máquina de lavar ...,"Gostaria de uma Máquina de lavar de 15kg, mas ...",NaN,NaN,NaN,1.0,NaN,1.0,0.476190
4,"Notebook lenovo, me parece bom, mas estou em d...",[Notebook Lenovo HP com o menor preço | Zoom C...,[Notebook Lenovo HP com o menor preço | Zoom C...,"Entendo sua dúvida! Ambas as marcas, Lenovo e ...","Notebook lenovo, me parece bom, mas estou em d...",NaN,NaN,NaN,NaN,NaN,1.0,0.087248
5,"Quero comprar um liquidificador, quais melhore...","[Walita Daily Turbo, que além de ter alta potê...",[0.5% de volta na loja todaCompare entre 22 lo...,Existem várias marcas de liquideificadores dis...,"Quero comprar um liquidificador, quais melhore...",NaN,NaN,0.808813,NaN,NaN,1.0,0.102326
6,"Quero um ar condicionado splir eletrolux, qual...",[Magazine LuizaControle Remoto para ar condici...,[0.5% de volta na loja todaCompare entre 10 lo...,"Infelizmente, não encontrei informações especí...","Quero um ar condicionado splir eletrolux, qual...",NaN,NaN,NaN,1.0,NaN,1.0,0.842105
7,Qual loja tem o menor valor do Samsung Galaxy ...,[Sansung Galaxy Z Fold7 com o menor preço | Zo...,[Sansung Galaxy Z Fold7 com o menor preço | Zo...,O Samsung Galaxy Z Fold7 com as especificações...,Qual loja tem o menor valor do Samsung Galaxy ...,NaN,NaN,0.768951,1.0,NaN,1.0,0.550000
8,Em qual loja vende aspirador de pó vertical da...,[WebcontinentalAspirador De Pó Vertical Britân...,[0.5% de volta na loja todaCompare entre 14 lo...,Vários produtos do tipo Aspirador De Pó Vertic...,Em qual loja vende aspirador de pó vertical da...,NaN,NaN,0.620133,NaN,NaN,1.0,0.777778
9,"Quero comprar um tv led LG de 43 polegadas, qu...","[TV LED 43"" LG 4K UA7500Menor preço via Amazon...",[Tv led LG 43 com o menor preço | Zoom Categor...,"De acordo com os dados disponíveis, o valor ma...","Quero comprar um tv led LG de 43 polegadas, qu...",NaN,NaN,NaN,1.0,NaN,1.0,0.785714


In [45]:
df.to_csv("dataset/evaluation.csv")

---

Medium Latency:

In [27]:
times = list()

for question in questions:
    begin = time()
    price_state = generate_price({
        'category': '', 
        'product': question['product'], 
        'query': question['query']
        })
    end = time() - begin
    times.append(end)

In [28]:
print("Medium latency:", sum(times) / len(times))

Medium latency: 63.65097672939301
